In [ ]:
from importlib import reload
reload(change)
reload(base)

In [ ]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [ ]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [ ]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [ ]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [ ]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [ ]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [ ]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [ ]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [ ]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [ ]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [ ]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [ ]:
all_pairs_of_amplitude=[]

In [ ]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change])) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

#### 1. Amplitude'u hesaplanan tüm pair'lar all_pairs_of_amplitude adlı dizide tutuluyor.  Hepsi tek DataFrame'de birleştirilmek istenirse aşağıdaki kod çalıştırılabilir.

In [ ]:
from functools import reduce

df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

#### 2. Henüz condition uygulanmamış sonuçlar excel'e yazdırabilir.

In [ ]:
df_mid_price.to_excel('out/mid_price.xlsx',)
df_spread.to_excel('out/spread.xlsx')
df_change.to_excel('out/change.xlsx')
df_final.to_excel('out/amplitude.xlsx')

# Koşula Göre Amplitude ve Duration Verisinin Yeniden Düzenlenmesi

In [34]:
#Önce condition modülümüzü import ediyoruz.
import src.condition as condition

In [39]:
# Hesaplayacağımız verileri saklayabilmek için boş bir dizi yarattık
condition_data = []

In [41]:
# Python'a ait lambda ifadelerini kullanabilmek için basit bir fonksiyon yazdık
def uygula(data):
    """parametre olarak gelen 'data' daki indeksi kaldırır.
       percentile(.6) ya göre koşul uygulandı.
       percentile silinip yerine medyan() yazılırsa, koşulu medyana göre hesaplar.
       ya da percentile'ın '.6' olan parametresi değiştirilebilir.
    """
    data = data.reset_index(drop=True)
    signs = condition.Sign(data).percentile(.6) #
    return condition.apply(data,signs)

In [68]:
# bir for döngüsü ile tüm pairlara erişiyoruz
for data in all_pairs_of_amplitude:
    #groupby içerisindeki floor metoduyla veriyi neye göre gruplayacağımızı belirtiyoruz.
    #Burada groupby 2 parametre aldı. ilk parametre ile 'güne göre', 2. parametre ile de 'saate' göre
    #ayırıp, 'uygula' metodunu her gruba ayrı ayrı uygulayacağını belirttik
    """ 'd' -> day
        '2h' -> 2şer saat
        'h'-> birer saat
        'nd' ->n güne böl
        'nh' -> n saate böl gibi        
    """ 
    """.apply metodu ile yapmak istediğimiz işlemi belirtiyoruz
        lambda x : uygula(x) -> her bir grubu sırayla
        x olarak adlandır ve uygula fonksiyonuna parametre olarak gönder
    """
    result = data.groupby([data.date.dt.floor('d'),data.date.dt.floor('2h')])\
        .apply(lambda x : uygula(x)).droplevel([0,1])
    #son olarak hesaplanan verileri condition_data adlı listeye atmasını söylüyoruz
    condition_data.append(result)

duration= {'pozitif': 7.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.8999999999809205e-05}
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 0.00010899999999969268, 'negatif': -9.649999999994385e-05}
duration= {'pozitif': 4.0, 'negatif': nan}
amplitude= {'pozitif': 0.00010899999999991472, 'negatif': -5.900000000014227e-05}
duration= {'pozitif': 12.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.900000000003125e-05}
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.8999999999809205e-05}
duration= {'pozitif': 6.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.0000000000105516e-05}
duration= {'pozitif': 8.0, 'negatif': nan}
amplitude= {'pozitif': 6.150000000004763e-05, 'negatif': -6.14999999999366e-05}
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999976694e-05, 'negatif': -6.14999999999366e-05}
dura

In [71]:
# percentile(0.6)'a göre hesaplanmış olan 1. pairi görüntüleyelim

In [72]:
condition_data[0]

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:13,-0.000218
1,2018-09-06 00:00:39,00:00:22,0.000050
2,2018-09-06 00:00:53,00:00:14,-0.000109
3,2018-09-06 00:01:01,00:00:08,0.000096
4,2018-09-06 00:01:22,00:00:21,-0.000159
...,...,...,...
347,2018-09-07 23:56:50,00:00:16,-0.000062
348,2018-09-07 23:57:25,00:00:35,0.000062
349,2018-09-07 23:58:50,00:01:25,-0.000200
350,2018-09-07 23:58:54,00:00:04,0.000111
